We'll be using differential dynamic microscopy. This technique was first described in 2008 in an article in PRL by Cerbino and Trappe. 
![PRL image](../Tutorials/DDM_Paper_PRL.PNG)

You can find the paper [here](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.100.188102). 

The heart of the DDM code (found in the `ddm_calc.py` file) is the computation of the image structure function. This is found by taking the average of the Fourier transforms of all image *differences*. By "image differences," I mean the result of subtracting two images separated by a given lag time $\Delta t$.

To describe the process mathematically, we find the difference between images separated by some lag time $\Delta t$:
$$\Delta I = I(x,y;t) - I(x,y;t + \Delta t)$$

For a given $\Delta t$ all such image differences are calculated. We then Fourier transform each $\Delta I$ and average all of the same $\Delta t$.

This results in the image structure function $D(q_x,q_y,\Delta t)$.

<div class="alert alert-block alert-info">
<b>Tip:</b> Use blue boxes (alert-info) for tips and notes. 
If it’s a note, you don’t have to include the word “Note”.
</div>

## Importing the necessary modules

### Modules for plotting
We use [matplotlib](https://matplotlib.org/) for creating figures and plots. Note that we use:
```python
%matplotlib inline
```
This seets the backend of matplotlib to `inline` which means the plots are included in the notebook. If you want the plots to also be interactive (e.g., having the ability to zoom, scroll, and save) then use:
```python
%matplotlib notebook
```

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

### Modules for numerical work
Here, we import [numpy](https://numpy.org/) and [xarray](https://xarray.pydata.org/en/stable/). 

Note that `xarray` might not have come with your Anaconda Python distribution (or whichever other distribution you installed). If that is the case, you'll need to [install](https://xarray.pydata.org/en/stable/getting-started-guide/installing.html) this package. 

In [ ]:
import numpy as np #numerical python used for working with arrays, mathematical operations
import xarray as xr #package for labeling and adding metadata to multi-dimensional arrays

### Import the PyDDM package
Make use you append to `sys.path` the directory containing the PyDDM code. 

In [ ]:
import sys
sys.path.append("../PyDDM") #must point to the PyDDM folder
import ddm_analysis_and_fitting as ddm

## Initiazing DDM_Analysis class and computing the DDM matrix
The instance of the `DDM_Analysis` class we create will need, when initialized, metadata about the images to analyze and the analysis and fitting parameters. This can be done by using a [yaml](https://yaml.org/) file as shown in the following cell of code (there, the metadata is saved in the file "*example_data_silica_beads.yml*". But one can also initialize `DDM_Analysis` with a dictionary containing the necessary metadata. One way to create such a dictionary and then using it to initialize `DDM_Analysis` is shown below. 
```python
import yaml
ddm_analysis_parameters_str = """
DataDirectory: 'C:/Users/rmcgorty/Documents/GitHub/DDM-at-USD/ExampleData/'
FileName: 'images_nobin_40x_128x128_8bit.tif'
Metadata:
  pixel_size: 0.242 # size of pixel in um
  frame_rate: 41.7 #frames per second
Analysis_parameters:
  number_lagtimes: 40
  last_lag_time: 600
  binning: no 
Fitting_parameters:
  model: 'DDM Matrix - Single Exponential' 
  Tau: [1.0, 0.001, 10]
  StretchingExp: [1.0, 0.5, 1.1]
  Amplitude: [1e2, 1, 1e6]
  Background: [2.5e4, 0, 1e7]
  Good_q_range: [5, 20]
  Auto_update_good_q_range: True
"""
parameters_as_dictionary = yaml.safe_load(ddm_analysis_parameters_str)
ddm_calc = ddm.DDM_Analysis(parameters_as_dictionary)
```

In [ ]:
ddm_calc = ddm.DDM_Analysis("example_data_silica_beads.yml")

In [ ]:
ddm_calc.calculate_DDM_matrix()

## Initiazing DDM_Fit class and fitting our data to a model

In [ ]:
ddm_fit = ddm.DDM_Fit(ddm_calc.data_yaml)

In [ ]:
ddm_calc.data_yaml

In [ ]:
print(ddm_fit)

In [ ]:
fit01 = ddm_fit.fit(name_fit = 'fit01')

In [ ]:
%matplotlib inline
ddm.fit_report(fit01, q_indices=[3,6,9,22], forced_qs=[4,16], use_new_tau=True, show=True)

### Trying a different mathematical model

In [ ]:
ddm_fit.reload_fit_model_by_name("ISF - Single Exponential")

In [ ]:
plt.figure()
plt.semilogx(ddm_fit.ddm_dataset.lagtime, ddm_fit.ddm_dataset.ddm_matrix[:,-1],'ro')
plt.title("Looking at highest q (typically, quickest to decay)")
plt.xlabel("Lag time (s)")
plt.ylabel("DDM Matrix")

In [ ]:
ddm_fit.ddm_dataset = ddm.recalculate_ISF_with_new_background(ddm_fit.ddm_dataset, 10)

In [ ]:
fit02 = ddm_fit.fit(name_fit = 'fit02', display_table=False)

In [ ]:
ddm.fit_report(fit02, q_indices=[3,6,9,22], forced_qs=[4,16], use_new_tau=True, show=True)

## Interactive with matplotlib

In [ ]:
%matplotlib notebook
fig, (ax, ax2) = plt.subplots(2, 1)
browser = ddm.Browse_DDM_Fits(fig, ax, ax2, fit02)

fig.canvas.mpl_connect('pick_event', browser.on_pick)
fig.canvas.mpl_connect('key_press_event', browser.on_press)

ax.set_title('Decay time vs wavevector')
ax.set_xlabel("q")
ax.set_ylabel("tau (s)")

## Saving the results

In [ ]:
ddm.save_fit_results_to_excel(fit01)

In [ ]:
fit01.to_netcdf("example_netcdf_file.nc")

In [ ]:
loaded_netcdf_file = xr.open_dataset("example_netcdf_file.nc")